In [2]:
import pandas as pd
#a = pd.read_csv("./the-movies-dataset/credits.csv")
b = pd.read_csv("./the-movies-dataset/movies_metadata.csv")

/Users/user1/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
c = b[['id','title','overview','tagline']]
d = c.dropna()

In [4]:
movies = d.to_dict('records')

In [5]:
for movie in movies:
    movie['overview']

In [6]:
import treetaggerwrapper as ttw
tagger = ttw.TreeTagger(TAGLANG='en')
tags = tagger.TagText('Hello, I am Yukie. I\'m from japan.')
print(tags)

['Hello\tUH\thello', ',\t,\t,', 'I\tPP\tI', 'am\tVBP\tbe', 'Yukie\tNP\tYukie', '.\tSENT\t.', 'I\tPP\tI', "'m\tVBP\tbe", 'from\tIN\tfrom', 'japan\tNN\tjapan', '.\tSENT\t.']


In [7]:
#movies_sample = movies[:10]
hinshilist = ['FW', 'JJ', 'JJR', 'NN', 'NNS', 'NP', 'NPS', 'VV', 'VVD', 'VVG', 'VVN', 'VVP', 'VVZ']

movie_word_dict = []
tagger = ttw.TreeTagger(TAGLANG='en')
for movie in movies:
    words = tagger.TagText(movie['overview'])
    wordslist = []
    for word in words:
        word_elements = word.split('\t')
        if len(word_elements)==3:
            hinshi = word_elements[1]
            word_ori = word_elements[2]
            if hinshi in hinshilist:
                wordslist.append(word_ori)
                movie_word_dict.append((movie['title'],word_ori))
    #print(wordslist)
    movie.update({'words':wordslist})

In [10]:
import networkx as nx
nodelist = []

G = nx.Graph()
for movie in movies:
    nodelist.append(movie['title'])
    nodelist.extend(movie['words'])
    G.add_nodes_from(movie['title'])
    G.add_nodes_from(movie['words'])

G.add_edges_from(movie_word_dict)

In [ ]:
import community
partition = community.best_partition(G)
size = float(len(set(partition.values())))
pos = nx.spring_layout(G)
count = 0.
for com in set(partition.values()):
    count += 1.
    list_nodes = [nodes for nodes in partition.keys() if partition[nodes] == com]
    nx.draw_networkx_nodes(G, pos, list_nodes, node_size=20, node_color = str(count/size) )
plt.show()
plt.savefig("movie_word_graph.png")